!pip install --upgrade yasa

# Artifact rejection

This notebook demonstrates how to perform automatic artifact rejection in YASA, specifically to detect major body movements on single or multi-channel data.

Please make sure to install the latest version of YASA first with: `pip install --upgrade yasa`

To start with, let's load a full night recording of EEG data (9 channels), sampled at 100 Hz and staged.

In [1]:
import yasa
import mne
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='darkgrid', font_scale=1.2)
import sys
import numpy as np
# np.set_printoptions(threshold=sys.maxsize)

In [2]:
ekgbr_channels=['EKG', 'Thorax', 'Abdomen']
emg_channels=['Leg-L', 'ARM-R', 'Mass2', 'Chin', 'Leg-R', 'ARM-L', 'Mass1']
eeg_channels=['F3', 'F4', 'A1', 'C3', 'C4', 'A2', 'O1', 'O2','F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
eog_channels=['LOC', 'ROC', 'LOC-1', 'ROC-1']
edf_file='/media/wapeul/2D33-43E11/ss/eeomg/220321hss/Traces_eeomg.edf'
csv_file='/media/wapeul/2D33-43E11/ss/eeomg/220321hss/hss-Events.csv'


In [3]:
def read_raw(edf_file, csv_file):
    # read raw data and anno( hypno), 
    raw = mne.io.read_raw_edf(edf_file,preload=True)#, eog = ['LOC-1', 'ROC-1']
    print('The channels are:', raw.ch_names)
    print('The sampling frequency is:', raw.info['sfreq'])
    df = pd.read_csv(csv_file)
    hypno =df['#Stage']#.to_numpy()
    start_frame = int(pd.read_csv(csv_file, nrows=1).columns[1])
    #     print(f"start_frame:{start_frame}, \nhypno:{hypno}")
    start_sec=int(start_frame/raw.info['sfreq'])
    end_sec=int(len(hypno)*30+start_frame/raw.info['sfreq'])
    #     print(f"start_sec:{start_sec}, end_sec:{end_sec}")
    raw.crop(tmin=start_sec, tmax=end_sec, include_tmax=True,verbose=True)
    # raw.drop_channels(['ROC-A1', 'LOC-A2', 'EMG1-EMG2', 'EKG-R-EKG-L'])
    raw.drop_channels(['EKG', 'Thorax', 'Abdomen', 'Leg-L', 'ARM-R', 'Mass2', 'Leg-R',\
                       'ARM-L', 'Mass1'])
    # raw.drop_channels(['LOC-1', 'ROC-1'])
    raw.drop_channels([ 'ROC-0', 'LOC-1', 'ROC-1','LOC-0', 'Chin' ])#'LOC-0',
    raw.drop_channels(['C4','O1', 'O2', 'F4'])# 'F3''C3',
    print('The channels are:', raw.ch_names)
    chan = raw.ch_names
    print(f"num of chan:{len(chan)},chan:{chan}")
    raw.resample(100)
    sf = raw.info['sfreq']
    raw.filter(0.3, 45);
    # raw.save('/media/wapeul/2D33-43E11/ss/eeomg/220321hss/hss.fif', overwrite=True)
    # raw = mne.io.read_raw_fif('/media/wapeul/2D33-43E11/ss/eeomg/220321hss/hss.fif')

    # Load data
    # f = np.load('data_full_6hrs_100Hz_9channels.npz')
    # Extract data, sf, and chan
    data = raw.get_data(units="uV")
    data = data/10000 # 예제는 수십 uV를 보여서 scale을 맞추려고
    sf = raw.info['sfreq']
    chan = raw.ch_names

    print('The channels are:', raw.ch_names)
    chan = raw.ch_names
    print(f"num of chan:{len(chan)},chan:{chan}")

    # Let's have a look at the data
    print('Chan =', chan)
    print('Sampling frequency =', sf, 'Hz')
    print('Data shape =', data.shape)
    print('Duration =', data.shape[1] / sf, 'seconds')

    # data= f['data']
    # chan = ['Fz', 'F3', 'F4', 'Cz', 'C3', 'C4', 'Pz', 'P3', 'P4']
    # sf = 100.
    # hypno = np.load('data_full_6hrs_100Hz_hypno.npz').get('hypno')
    times = np.arange(data.size) / sf
    print(data.shape)
    print(chan)
    print(np.round(data[:, 0:5], 3))
    return hypno, raw, data

In [4]:
def apply_artifacts(hypno,raw, data, ch_name='C3-A2', patient_name='None'):

    # apply artifacts

    # upsample하는 이유는 visual inspection 및 뒤에나오는 artifact들과 어떻게 다른지확인
#     yasa.plot_hypnogram(hypno)
    hypno_up = yasa.hypno_upsample_to_data(hypno, sf_hypno=1/30, data=raw)
    # visual inspection
    sf=100
#     for j in range(len(raw.ch_names)):
#         print("channel check:",raw.ch_names[j])
#         yasa.plot_spectrogram(data[j,:], sf, hypno_up);#'F3', F3 데이터는 깨진듯

    # window = 30 으로 맞춘이유는 hypno에서 masking filter해서 training 하려고
    art, zscores = yasa.art_detect(data, sf, window=30, method='covar', threshold=3)# orignally win=5
    # art.shape, zscores.shape, art.shape[0] # ((3928,), (3928,), 3928)
    # Art is an aray of 0 and 1, where 0 indicates a clean (or good epoch)  and 1 indicates an artifact epoch
    # print(art)
    print(f'{art.sum()} / {art.size} epochs rejected.')
    # Plot the artifact vector
#     plt.plot(art);
#     plt.yticks([0, 1], labels=['Good (0)', 'Art (1)']);
    # z distribution check
#     sns.distplot(zscores)
#     plt.title('Histogram of z-scores')
#     plt.xlabel('Z-scores')
#     plt.ylabel('Density')
#     plt.axvline(3, color='r', label='Threshold')
#     plt.axvline(-3, color='r')
#     plt.legend(frameon=False);
    from scipy.special import erf
    threshold = 3
    perc_expected_rejected = (1 - erf(threshold / np.sqrt(2))) * 100
    print(f'{perc_expected_rejected:.2f}% of all epochs are expected to be rejected.')
    # Actual
    # (art.sum() / art.size) * 100
    print(f'{(art.sum() / art.size) * 100:.2f}% of all epochs were actually rejected.')
    # merge artifact now with hypno
    # Now that `hypno` and `art_up` have the same size, we can easily merge them. Below, we mark as artifact (code = -1) all the 5-seconds epochs that were automatically rejected:
    # The resolution of art is 5 seconds, so its sampling frequency is 1/5 
    # (= 0.2 Hz), 우리의 sleep event는 대체로 30 sec이므로 그냥 참고만
    sf_art = 1 / 5
    art_up = yasa.hypno_upsample_to_data(art, sf_art, data, sf)
    assert art_up.shape==hypno_up.shape

    # Add -1 to hypnogram where artifacts were detected
    hypno_with_art = hypno_up.copy()
    hypno_with_art[art_up] = -1

    # Proportion of each stage in ``hypno_with_art``
    pd.Series(hypno_with_art).value_counts(normalize=True)
    print("PSD Good or Bad with visual inspection")
#     yasa.plot_spectrogram(data[0, :], sf, hypno_with_art); 
    # hss data 0은 그려보면, 20Hz부근에 센서 오류와 2hr근처에서 앞부분과 뒷부분이
    # 확연이 차이가 난다.
    # 세로 artifact들도 보인다. 이 sensor channel은 reference로 사용은 어렵겠다.
    # 반면 channel index 6인 'C3-A2'는 절단은 안보이지만 세로 artifact들은 보인다.
#     yasa.plot_spectrogram(data[raw.ch_names.index(ch_name),:], sf, hypno_with_art);

    # Plot new hypnogram and spectrogram on Fz
#     yasa.plot_spectrogram(data[6, :], sf, hypno_with_art);
    # Apply the artifact rejection with method='std'
    # In order to have more details on the artifact rejection, one can set the
    # logging level to INFO
    art_std, zscores_std = yasa.art_detect(data, sf, window=30, method='std', threshold=3, verbose='info')
    # art_std.shape, zscores_std.shape
    # We can average the z-scores across all channels
    avg_zscores = zscores_std.mean(-1)
    # avg_zscores.shape
    # And check the distribution
#     sns.distplot(avg_zscores)
#     plt.title('Histogram of z-scores')
#     plt.xlabel('Z-scores')
#     plt.ylabel('Density')
#     plt.axvline(3, color='r', label='Threshold')
#     plt.axvline(-3, color='r')
#     plt.legend(frameon=False);
    print(f'The correlation between the two methods is r = {np.corrcoef(art, art_std)[0, 1]:.2f}')

    # 여기부터는 hypnogram의 hypno_up을 활용해서 art 및 zscore를 빼냄
    # covariance로 artififact filter
    art, zscores = yasa.art_detect(data, sf, window=30, hypno=hypno_up, include=(1, 2, 3, 4), 
                                   method='covar', threshold=3, verbose='info')
    # art.shape, zscores.shape
    # 그려봄 for drawing synchronizing with hypno and raw data, we need upsample 100Hz
    art_up = yasa.hypno_upsample_to_data(art, sf_art, data, sf)
    # Add -1 to hypnogram where artifacts were detected
    hypno_with_art = hypno_up.copy()
    hypno_with_art[art_up] = -1
#     yasa.plot_spectrogram(data[6, :], sf, hypno_with_art);

    # std로 artfifact으로 filter
    art_std, zscores_std = yasa.art_detect(data, sf, window=30, hypno=hypno_up, include=(1, 2, 3, 4), 
                                           method='std', threshold=3, verbose='info')
    print(f"art_std.shape:{art_std.shape}, zscores_std.shape:{zscores_std.shape}")
    # hypno와 artifact masking을 합침 > 그림으로 그리려고
    art_up = yasa.hypno_upsample_to_data(art_std, sf_art, data, sf)

    # Add -1 to hypnogram where artifacts were detected
    hypno_with_art = hypno_up.copy()
    hypno_with_art[art_up] = -1

#     yasa.plot_spectrogram(data[6, :], sf, hypno_with_art);# yasa에서는 Cz 추천함
    # hypno[art]=-1 그림 그릴때만, filter할때는 index filter
    # `n_chan_reject = 2` or 3, 4 ?
    art_std, zscores_std = yasa.art_detect(data, sf, window=30, hypno=hypno_up, include=(1, 2, 3, 4), 
                                           method='std', threshold=3, n_chan_reject=2, verbose='info')

    sls = yasa.SleepStaging(raw, eeg_name=ch_name)
    hypno_pred = sls.predict()  # Predict the sleep stages
    hypno_pred = yasa.hypno_str_to_int(hypno_pred)  # Convert "W" to 0, "N1" to 1, etc
    hypno_pred_art = hypno_pred[~art]
    hypno_art=hypno[~art]
    assert hypno_art.shape==hypno_pred_art.shape
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
#     yasa.plot_hypnogram(hypno_pred_art);  # Plot
    accuracy= accuracy_score(hypno_art, hypno_pred_art)
    print(f"{patient_name},{ch_name}:The accuracy is {100 * accuracy:.3f}%")
    # confusion_matrix(hypno, hypno_pred)
    return ch_name, accuracy


In [5]:
edf_dir='/media/wapeul/2D33-43E11/ss/eeomg'
channel_names=['F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
import os
import re
# accuracy 결과 저장용 dataframe 만들기
result_predict_art = pd.DataFrame(columns=['patient_name', 'channel', 'acc_score'])
subfolders = [ f.path for f in os.scandir(edf_dir) if f.is_dir() ]
print(f"subfolders:{subfolders}")
count=0
# if subfolder
for i,v in enumerate(subfolders): # each folders
    print(f"Count:{count}")
    folders = edf_dir+v

    ## 환자 측정일자 및 이름 분리하기
    s=v.split('/')
    print("s:",s[6])
    match = re.match(r"([0-9]+)([a-z]+)", s[6], re.I)
    if match:
        items = match.groups()
    print("items:",items)
    patient_name = items[1]
    
    # 가독성 높이려 타이틀 추가
    print("\n\n")
    print("++++++++++++++++++++++" + "+"*len(patient_name))
    print(i,". Patient Name : ", patient_name)
    print("++++++++++++++++++++++" + "+"*len(patient_name))
    
    # 주어진 데이터에서 13번째 디렉토리는 어떤 채널을 선택해도 에러 발생해 임의로 제외시킴 (일반화 대상 아님)
#     if i == 12:
#         print("\nchannel not in list error 발생으로 임의 제외함\n")
#         continue
        
    #print(f"v:{v}")
    files=os.listdir(v)    
    
    #print(f"files:{files}")
    for file in files:
        print(f"file:{file}")
        if file.endswith(".csv"):
            csv_file = v+'/'+file
        elif file.endswith("g.edf"):
            edf_file = v+'/'+file
    #print(f"csv_file:{csv_file}")
    #print(f"edf_file:{edf_file}")
    hypno, raw, data= read_raw(edf_file, csv_file)
    for i in channel_names:
        # 환자 이름 추가하여 함수에 입력
        # spectrogram 에 사용한 channel 명과 acc_score 를 받음
        channel_name, acc_score = apply_artifacts(hypno, raw, data, ch_name=i, patient_name=patient_name)
        # accuracy 를 전체적으로 확인할 수 있도록 dataframe 에 추가
        result_predict_art.loc[count] = [patient_name, channel_name, acc_score]  
        count=count+1
    print(f"---------------------{patient_name},{i}-------------")

subfolders:['/media/wapeul/2D33-43E11/ss/eeomg/220321hss', '/media/wapeul/2D33-43E11/ss/eeomg/220329lsb', '/media/wapeul/2D33-43E11/ss/eeomg/220629hsj', '/media/wapeul/2D33-43E11/ss/eeomg/220701kyj', '/media/wapeul/2D33-43E11/ss/eeomg/220706pmr', '/media/wapeul/2D33-43E11/ss/eeomg/220708jwh', '/media/wapeul/2D33-43E11/ss/eeomg/220709nmh', '/media/wapeul/2D33-43E11/ss/eeomg/220713kkc', '/media/wapeul/2D33-43E11/ss/eeomg/220715yjh', '/media/wapeul/2D33-43E11/ss/eeomg/220718so', '/media/wapeul/2D33-43E11/ss/eeomg/220721kks', '/media/wapeul/2D33-43E11/ss/eeomg/220725pms', '/media/wapeul/2D33-43E11/ss/eeomg/220802lig', '/media/wapeul/2D33-43E11/ss/eeomg/220808pej', '/media/wapeul/2D33-43E11/ss/eeomg/220811sms', '/media/wapeul/2D33-43E11/ss/eeomg/220817ljw', '/media/wapeul/2D33-43E11/ss/eeomg/220819hbc', '/media/wapeul/2D33-43E11/ss/eeomg/220822sjs', '/media/wapeul/2D33-43E11/ss/eeomg/220823kjs', '/media/wapeul/2D33-43E11/ss/eeomg/220824lys', '/media/wapeul/2D33-43E11/ss/eeomg/220826msh', '/

/tmp/ipykernel_999995/3716747382.py:3: RuntimeWarning: Channel names are not unique, found duplicates for: {'LOC', 'ROC'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file,preload=True)#, eog = ['LOC-1', 'ROC-1']


The channels are: ['EKG', 'Leg-L', 'ARM-R', 'Mass2', 'Chin', 'Leg-R', 'ARM-L', 'Mass1', 'F3', 'F4', 'LOC-0', 'ROC-0', 'A1', 'C3', 'C4', 'A2', 'O1', 'O2', 'Thorax', 'Abdomen', 'F3-A2', 'F4-A1', 'LOC-1', 'ROC-1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
The sampling frequency is: 200.0
The channels are: ['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
num of chan:10,chan:['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 47.50 Hz)
- 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s finished


The channels are: ['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
num of chan:10,chan:['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
Chan = ['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
Sampling frequency = 100.0 Hz
Data shape = (10, 1944000)
Duration = 19440.0 seconds
(10, 1944000)
['F3', 'A1', 'C3', 'A2', 'F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']
[[  0.    -12.423  32.507 145.731 198.857]
 [  0.    -20.56  -24.699  -1.819   8.804]
 [  0.    -47.454 -23.572  70.22   98.81 ]
 [ -0.     26.699  24.167  13.048  -5.266]
 [  0.    -36.413  -3.876 116.023 212.828]
 [  0.    -19.966  49.643 177.893 221.742]
 [  0.    -61.398 -54.954  44.514 117.063]
 [  0.      9.582  67.972 143.323 167.74 ]
 [  0.    -19.314 -89.477 -85.699  -1.87 ]
 [  0.     64.939  39.143 -22.712 -70.852]]


...

02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN000 = 158.1283 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN000 = 10471.6774 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN001 = 49.9288 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN001 = 11274.7185 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN002 = 82.1500 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN002 = 20690.0777 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN003 = 49.5243 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN003 = 11198.7726 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN004 = 87.8487 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN004 = 7084.3392 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of CHAN005 = 187.7532 uV
02-Oct-22 04:57:09 | INFO | Peak-to-peak amplitude of CHAN005 = 46660.9312 uV
02-Oct-22 04:57:09 | INFO | Trimmed standard deviation of

art_std.shape:(733,), zscores_std.shape:(733, 10)


02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN000 = 158.1283 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN000 = 10471.6774 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN001 = 49.9288 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN001 = 11274.7185 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN002 = 82.1500 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN002 = 20690.0777 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN003 = 49.5243 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN003 = 11198.7726 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN004 = 87.8487 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN004 = 7084.3392 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of CHAN005 = 187.7532 uV
02-Oct-22 04:57:11 | INFO | Peak-to-peak amplitude of CHAN005 = 46660.9312 uV
02-Oct-22 04:57:11 | INFO | Trimmed standard deviation of

sms,C3-A2:The accuracy is 52.468%
---------------------sms,C3-A2-------------


/home/wapeul/anaconda3/envs/yasa2/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
result_predict_art.to_csv('cha_predict_art.csv')

We can also apply a more conservative detection by specifying the number of channels that must be above or below threshold for the epoch to be marked as an artifact. In the code below, `n_chan_reject = 2` means that at least 2 channels must be above or below the threshold for the epoch to be marked as an artifact.